In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import pickle
import PIL.Image as Image
from torchvision.datasets.folder import ImageFolder
import torch.nn.functional as F
from pytorch_msssim import ssim
from skimage import measure
import time

In [2]:
class LeNet(nn.Module):
    def __init__(self, channel=3, hideen=768, num_classes=4):
        super(LeNet, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Conv2d(channel, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=1),
            act(),
        )
        self.fc = nn.Sequential(
            nn.Linear(hideen, num_classes)
        )

    def forward(self, x):
        out = self.body(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
    
def weights_init(m):
    if hasattr(m, "weight"):
        m.weight.data.uniform_(-0.5, 0.5)
    if hasattr(m, "bias"):
        m.bias.data.uniform_(-0.5, 0.5)

# def weights_init(m):
#     try:
#         if hasattr(m, "weight"):
#             m.weight.data.uniform_(-0.5, 0.5)
#     except Exception:
#         print('warning: failed in weights_init for %s.weight' % m._get_name())
#     try:
#         if hasattr(m, "bias"):
#             m.bias.data.uniform_(-0.5, 0.5)
#     except Exception:
#         print('warning: failed in weights_init for %s.bias' % m._get_name())



In [3]:

dataset = 'Brian_MRI'
root_path = '.'
data_path = os.path.join(root_path, './data').replace('\\', '/')
save_path = os.path.join(root_path, 'results/iDLG_%s'%dataset).replace('\\', '/')



''' load data '''
if dataset == 'MNIST':
    shape_img = (28, 28)
    num_classes = 10
    channel = 1
    hidden = 588
    dst = datasets.MNIST(data_path, download=False)


elif dataset == 'Brian_MRI':
    shape_img = (32, 32)
    num_classes = 4
    channel = 3
    hidden = 768
    dst = ImageFolder(r'C:\Users\badha\OneDrive - Florida International University\Desktop\PhD at FIU\Solid Lab\Spring 2023\CPL Attack Paper Exp\Brain-Tumor-MRI-Dataset\Training')



else:
    exit('unknown dataset')



lr = 1.0
num_dummy = 1
Iteration = 300
num_exp = 100

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

tt = transforms.Compose([transforms.Resize(32),transforms.CenterCrop(32),transforms.ToTensor()])




tp = transforms.Compose([transforms.ToPILImage()])

print(dataset, 'root_path:', root_path)
print(dataset, 'data_path:', data_path)
print(dataset, 'save_path:', save_path)

if not os.path.exists('results'):
    os.mkdir('results')
if not os.path.exists(save_path):
    os.mkdir(save_path)







Brian_MRI root_path: .
Brian_MRI data_path: ././data
Brian_MRI save_path: ./results/iDLG_Brian_MRI


In [4]:
ASR=0
whole_time=0
sSim=0
mSe=0
count=0

''' train DLG and iDLG '''
for idx_net in range(num_exp):
    net = LeNet(channel=channel, hideen=hidden, num_classes=num_classes)
    net.apply(weights_init)

    print('running %d|%d experiment'%(idx_net, num_exp))
    net = net.to(device)
    idx_shuffle = np.random.permutation(len(dst))
    #idx_no_shuffle = np.arange(len(dst))
    start=time.process_time()
    for method in ['iDLG']:#, 'iDLG']:
        print('%s, Try to generate %d images' % (method, num_dummy))

        criterion = nn.CrossEntropyLoss().to(device)
        imidx_list = []

        for imidx in range(num_dummy):
            idx = idx_shuffle[imidx]
            imidx_list.append(idx)
            tmp_datum = tt(dst[idx][0]).float().to(device)
            tmp_datum = tmp_datum.view(1, *tmp_datum.size())
            tmp_label = torch.Tensor([dst[idx][1]]).long().to(device)
            tmp_label = tmp_label.view(1, )
            if imidx == 0:
                gt_data = tmp_datum
                gt_label = tmp_label
            else:
                gt_data = torch.cat((gt_data, tmp_datum), dim=0)
                gt_label = torch.cat((gt_label, tmp_label), dim=0)


        # compute original gradient
        out = net(gt_data)
        y = criterion(out, gt_label)
        dy_dx = torch.autograd.grad(y, net.parameters())
        original_dy_dx = list((_.detach().clone() for _ in dy_dx))

        # generate dummy data and label
        dummy_data = torch.randn(gt_data.size()).to(device).requires_grad_(True)
        dummy_label = torch.randn((gt_data.shape[0], num_classes)).to(device).requires_grad_(True)

        if method == 'DLG':
            optimizer = torch.optim.LBFGS([dummy_data, dummy_label])
        elif method == 'iDLG':
            optimizer = torch.optim.LBFGS([dummy_data, ])
            # predict the ground-truth label
            label_pred = torch.argmin(torch.sum(original_dy_dx[-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)

        history = []
        history_iters = []
        losses = []
        mses = []
        mse_f = []
        SSIMS = []
        train_iters = []

        print('lr =', lr)
        for iters in range(Iteration):

            def closure():
                optimizer.zero_grad()
                pred = net(dummy_data)
                if method == 'DLG':
                    dummy_loss = - torch.mean(torch.sum(torch.softmax(dummy_label, -1) * torch.log(torch.softmax(pred, -1)), dim=-1))
                    #dummy_loss = criterion(pred, gt_label)
                elif method == 'iDLG':
                    dummy_loss = criterion(pred, label_pred)

                dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)

                grad_diff = 0
                for gx, gy in zip(dummy_dy_dx, original_dy_dx):
                    grad_diff += ((gx - gy) ** 2).sum()
                grad_diff.backward()
                return grad_diff

            optimizer.step(closure)
            current_loss = closure().item()
            train_iters.append(iters)
            losses.append(current_loss)
            mses.append(torch.mean((dummy_data-gt_data)**2).item())
            mse_f.append(F.mse_loss(dummy_data,torch.unsqueeze(gt_data,dim=0),  reduction='mean').item())
            SSIMS.append(ssim(dummy_data,gt_data,data_range=0).item())


            if iters % int(Iteration / 30) == 0:
                current_time = str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))
                print(current_time, iters, 'loss = %.8f, mse = %.8f, mse_with_function = %.8f, ssim = %.8f' %(current_loss, mses[-1], mse_f[-1], SSIMS[-1]))
                #print(current_time, iters, 'loss = %.8f, mse = %.8f, mse_with_function = %.8f' %(current_loss, mses[-1], mse_f[-1]))
                history.append([tp(dummy_data[imidx].cpu()) for imidx in range(num_dummy)])
                history_iters.append(iters)

                for imidx in range(num_dummy):
                    #plt.figure(figsize=(12, 8))
                    #plt.subplot(3, 10, 1)
                    plt.imshow(tp(gt_data[imidx].cpu()))
                    for i in range(min(len(history), 29)):
                        #plt.subplot(3, 10, i + 2)
                        plt.imshow(history[i][imidx])
                        #plt.title('iter=%d' % (history_iters[i]))
                        plt.axis('off')
                    if method == 'DLG':
                        #plt.savefig('%s/DLG_on_%s_%05d.png' % (save_path, imidx_list, imidx_list[imidx]))
                        plt.close()
                    elif method == 'iDLG':
                        plt.savefig('%s/Lbl_%s_iDLG_on_%s_%05d.png' % (save_path, gt_label.item(), imidx_list, imidx_list[imidx]))
                        plt.close()

                if current_loss < 0.000001: # converge
                    break

        if method == 'DLG':
            loss_DLG = losses
            label_DLG = torch.argmax(dummy_label, dim=-1).detach().item()
            mse_DLG = mses
            mse_f_iDLG = mse_f
            SSIM_DLG = SSIMS
        elif method == 'iDLG':
            loss_iDLG = losses
            label_iDLG = label_pred.item()
            mse_iDLG = mses
            mse_f_iDLG = mse_f
            SSIM_iDLG = SSIMS

    end=time.process_time()

    print('imidx_list:', imidx_list)
#         print('loss_DLG:', loss_DLG[-1], 'loss_iDLG:', loss_iDLG[-1])
#         print('mse_DLG:', mse_DLG[-1], 'mse_iDLG:', mse_iDLG[-1])
#         print('gt_label:', gt_label.detach().cpu().data.numpy(), 'lab_DLG:', label_DLG, 'lab_iDLG:', label_iDLG)

    
    #print('loss_DLG:', loss_DLG[-1], 'loss_iDLG:', loss_iDLG[-1])
    #print('mse_DLG:', mse_DLG[-1], 'mse_iDLG:', mse_iDLG[-1])
    print('loss_iDLG:', loss_iDLG[-1])
    print('mse_iDLG:', mse_iDLG[-1])
    print('mse_iDLG_with_functuon:', mse_f_iDLG[-1])
    mSe=mSe+mse_f_iDLG[-1]
    print('SSIM_iDLG:', SSIM_iDLG[-1])
    sSim=sSim+SSIM_iDLG[-1]
    duration=end-start
    print('Duration:',duration)
    whole_time=whole_time+duration
    print('gt_label:', gt_label.detach().cpu().data.numpy(), 'lab_iDLG:', label_iDLG)
    
    print('----------------------\n\n')
    
    if(SSIM_iDLG[-1]>=.90):
        ASR=ASR+1
        #print(AS)
    count=count+1
#     if(mse_f_iDLG[-1]>100 or mse_DLG[-1]>100):
#         break




running 0|100 experiment
iDLG, Try to generate 1 images
lr = 1.0
[2024-03-21 14:37:02] 0 loss = 0.00011419, mse = 1.03190267, mse_with_function = 1.03190267, ssim = 0.00091095


C:\Users\badha\AppData\Local\Temp\ipykernel_15932\2191301953.py:88: UserWarning: Using a target size (torch.Size([1, 1, 3, 32, 32])) that is different to the input size (torch.Size([1, 3, 32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_f.append(F.mse_loss(dummy_data,torch.unsqueeze(gt_data,dim=0),  reduction='mean').item())


[2024-03-21 14:37:03] 10 loss = 0.00011235, mse = 1.03190064, mse_with_function = 1.03190064, ssim = 0.00091131
[2024-03-21 14:37:05] 20 loss = 0.00011035, mse = 1.03189898, mse_with_function = 1.03189898, ssim = 0.00091167
[2024-03-21 14:37:06] 30 loss = 0.00010825, mse = 1.03189659, mse_with_function = 1.03189659, ssim = 0.00091204
[2024-03-21 14:37:07] 40 loss = 0.00010604, mse = 1.03189445, mse_with_function = 1.03189445, ssim = 0.00091241
[2024-03-21 14:37:08] 50 loss = 0.00010364, mse = 1.03189230, mse_with_function = 1.03189230, ssim = 0.00091280
[2024-03-21 14:37:10] 60 loss = 0.00010115, mse = 1.03189015, mse_with_function = 1.03189015, ssim = 0.00091318
[2024-03-21 14:37:11] 70 loss = 0.00009844, mse = 1.03188801, mse_with_function = 1.03188801, ssim = 0.00091360
[2024-03-21 14:37:12] 80 loss = 0.00009561, mse = 1.03188610, mse_with_function = 1.03188610, ssim = 0.00091404
[2024-03-21 14:37:13] 90 loss = 0.00009257, mse = 1.03188491, mse_with_function = 1.03188491, ssim = 0.0

[2024-03-21 14:43:53] 120 loss = 0.00003559, mse = 0.00526406, mse_with_function = 0.00526406, ssim = 0.85425198
[2024-03-21 14:44:04] 130 loss = 0.00002031, mse = 0.00332341, mse_with_function = 0.00332341, ssim = 0.90036154
[2024-03-21 14:44:14] 140 loss = 0.00001330, mse = 0.00234672, mse_with_function = 0.00234672, ssim = 0.92686772
[2024-03-21 14:44:24] 150 loss = 0.00000889, mse = 0.00164293, mse_with_function = 0.00164293, ssim = 0.94689769
[2024-03-21 14:44:33] 160 loss = 0.00000669, mse = 0.00129821, mse_with_function = 0.00129821, ssim = 0.95697755
[2024-03-21 14:44:42] 170 loss = 0.00000476, mse = 0.00095990, mse_with_function = 0.00095990, ssim = 0.96749628
[2024-03-21 14:44:53] 180 loss = 0.00000343, mse = 0.00071415, mse_with_function = 0.00071415, ssim = 0.97554845
[2024-03-21 14:45:00] 190 loss = 0.00000285, mse = 0.00058990, mse_with_function = 0.00058990, ssim = 0.97972429
[2024-03-21 14:45:01] 200 loss = 0.00000285, mse = 0.00058990, mse_with_function = 0.00058990, s

[2024-03-21 14:52:10] 0 loss = 27.52175713, mse = 1.07532144, mse_with_function = 1.07532144, ssim = 0.01206821
[2024-03-21 14:52:20] 10 loss = 0.71648246, mse = 0.39775270, mse_with_function = 0.39775270, ssim = 0.12835531
[2024-03-21 14:52:30] 20 loss = 0.08879364, mse = 0.13889953, mse_with_function = 0.13889953, ssim = 0.30442685
[2024-03-21 14:52:41] 30 loss = 0.01550116, mse = 0.04793962, mse_with_function = 0.04793962, ssim = 0.50945878
[2024-03-21 14:52:52] 40 loss = 0.00345149, mse = 0.01834846, mse_with_function = 0.01834846, ssim = 0.70284432
[2024-03-21 14:53:03] 50 loss = 0.00104169, mse = 0.00752363, mse_with_function = 0.00752363, ssim = 0.82897818
[2024-03-21 14:53:13] 60 loss = 0.00036186, mse = 0.00299612, mse_with_function = 0.00299612, ssim = 0.90651929
[2024-03-21 14:53:24] 70 loss = 0.00012974, mse = 0.00116283, mse_with_function = 0.00116283, ssim = 0.94676530
[2024-03-21 14:53:35] 80 loss = 0.00004779, mse = 0.00042545, mse_with_function = 0.00042545, ssim = 0.9

...

imidx_list: [3882]
loss_iDLG: 30.660213470458984
mse_iDLG: 42265993216.0
mse_iDLG_with_functuon: 42265993216.0
SSIM_iDLG: -2.784120156640313e-12
Duration: 14.671875
gt_label: [2] lab_iDLG: 2
----------------------


running 88|100 experiment
iDLG, Try to generate 1 images
lr = 1.0
[2024-03-21 17:14:07] 0 loss = 42.25983429, mse = 1.05404639, mse_with_function = 1.05404639, ssim = 0.00313122
[2024-03-21 17:14:12] 10 loss = 1.46791899, mse = 0.52735001, mse_with_function = 0.52735001, ssim = 0.01321961
[2024-03-21 17:14:18] 20 loss = 0.25166333, mse = 0.28328305, mse_with_function = 0.28328305, ssim = 0.03301061
[2024-03-21 17:14:24] 30 loss = 0.07424962, mse = 0.16063000, mse_with_function = 0.16063000, ssim = 0.07290643
[2024-03-21 17:14:30] 40 loss = 0.02806954, mse = 0.09435473, mse_with_function = 0.09435473, ssim = 0.11433350
[2024-03-21 17:14:36] 50 loss = 0.01135022, mse = 0.05420091, mse_with_function = 0.05420091, ssim = 0.17747931
[2024-03-21 17:14:43] 60 loss = 0.00486562, ms

[2024-03-21 17:16:51] 10 loss = 0.58448470, mse = 0.62072444, mse_with_function = 0.62072444, ssim = 0.01942424
[2024-03-21 17:16:57] 20 loss = 0.13035583, mse = 0.36444002, mse_with_function = 0.36444002, ssim = 0.04434243
[2024-03-21 17:17:04] 30 loss = 0.04149159, mse = 0.21930724, mse_with_function = 0.21930724, ssim = 0.07997321
[2024-03-21 17:17:10] 40 loss = 0.01421864, mse = 0.11669552, mse_with_function = 0.11669552, ssim = 0.14008717
[2024-03-21 17:17:16] 50 loss = 0.00524406, mse = 0.06123146, mse_with_function = 0.06123146, ssim = 0.21636528
[2024-03-21 17:17:23] 60 loss = 0.00190716, mse = 0.03091688, mse_with_function = 0.03091688, ssim = 0.32036084
[2024-03-21 17:17:29] 70 loss = 0.00091356, mse = 0.01851997, mse_with_function = 0.01851997, ssim = 0.40669781
[2024-03-21 17:17:36] 80 loss = 0.00042610, mse = 0.01058752, mse_with_function = 0.01058752, ssim = 0.51165950
[2024-03-21 17:17:42] 90 loss = 0.00021292, mse = 0.00620258, mse_with_function = 0.00620258, ssim = 0.6

[2024-03-21 17:19:42] 40 loss = 0.00046426, mse = 0.00164598, mse_with_function = 0.00164598, ssim = 0.89680523
[2024-03-21 17:19:48] 50 loss = 0.00009638, mse = 0.00043273, mse_with_function = 0.00043273, ssim = 0.96973747
[2024-03-21 17:19:54] 60 loss = 0.00002364, mse = 0.00010577, mse_with_function = 0.00010577, ssim = 0.99281734
[2024-03-21 17:20:00] 70 loss = 0.00000580, mse = 0.00002401, mse_with_function = 0.00002401, ssim = 0.99834865
[2024-03-21 17:20:07] 80 loss = 0.00000123, mse = 0.00000476, mse_with_function = 0.00000476, ssim = 0.99965858
[2024-03-21 17:20:14] 90 loss = 0.00000032, mse = 0.00000130, mse_with_function = 0.00000130, ssim = 0.99989939
imidx_list: [3289]
loss_iDLG: 3.199336333636893e-07
mse_iDLG: 1.296577920584241e-06
mse_iDLG_with_functuon: 1.296577920584241e-06
SSIM_iDLG: 0.9998993873596191
Duration: 55.640625
gt_label: [2] lab_iDLG: 2
----------------------


running 93|100 experiment
iDLG, Try to generate 1 images
lr = 1.0
[2024-03-21 17:20:14] 0 loss = 

[2024-03-21 17:25:53] 20 loss = 0.03677246, mse = 0.05311245, mse_with_function = 0.05311245, ssim = 0.16483247
[2024-03-21 17:25:59] 30 loss = 0.00640705, mse = 0.01736660, mse_with_function = 0.01736660, ssim = 0.35125145
[2024-03-21 17:26:05] 40 loss = 0.00138497, mse = 0.00621812, mse_with_function = 0.00621812, ssim = 0.54105037
[2024-03-21 17:26:11] 50 loss = 0.00043679, mse = 0.00260522, mse_with_function = 0.00260522, ssim = 0.70343632
[2024-03-21 17:26:17] 60 loss = 0.00015802, mse = 0.00108786, mse_with_function = 0.00108786, ssim = 0.80328262
[2024-03-21 17:26:23] 70 loss = 0.00005957, mse = 0.00043060, mse_with_function = 0.00043060, ssim = 0.87396276
[2024-03-21 17:26:30] 80 loss = 0.00002260, mse = 0.00016677, mse_with_function = 0.00016677, ssim = 0.92280102
[2024-03-21 17:26:36] 90 loss = 0.00000830, mse = 0.00006229, mse_with_function = 0.00006229, ssim = 0.95478523
[2024-03-21 17:26:42] 100 loss = 0.00000286, mse = 0.00002203, mse_with_function = 0.00002203, ssim = 0.

[2024-03-21 17:29:18] 100 loss = 0.00000062, mse = 0.00000247, mse_with_function = 0.00000247, ssim = 0.99987078
imidx_list: [1365]
loss_iDLG: 6.204560349942767e-07
mse_iDLG: 2.4718935947021237e-06
mse_iDLG_with_functuon: 2.4718935947021237e-06
SSIM_iDLG: 0.999870777130127
Duration: 60.75
gt_label: [1] lab_iDLG: 1
----------------------




In [11]:
print("Count:",count)
print("ASR:",ASR/count)
print("Avg. SSIM:",sSim/count)
print("Avg. MSE is:",mSe/count)
print("Avg. time:",whole_time/count)

Count: 100
ASR: 0.74
Avg. SSIM: 0.7455592972056979
Avg. MSE is: 6213765879961.534
Avg. time: 102.52078125


In [6]:
torch.mean((dummy_data-gt_data)**2).item()

2.4718935947021237e-06

In [7]:
 mse_iDLG[-1]

2.4718935947021237e-06

In [8]:
imidx_list

[1365]

In [9]:
dummy_data

tensor([[[[ 8.2723e-05,  1.6970e-04,  4.8938e-04,  ..., -2.6551e-03,
           -1.9024e-03,  3.5224e-03],
          [-6.6493e-05, -3.3247e-04, -5.2981e-04,  ...,  1.5349e-03,
            2.8292e-03, -8.4765e-04],
          [-6.0793e-05,  4.1317e-04, -4.0093e-05,  ...,  5.0058e-05,
           -2.2643e-03, -1.9725e-04],
          ...,
          [ 2.6263e-04,  1.1276e-03,  1.3940e-04,  ...,  3.1213e-04,
            4.1930e-03, -6.5259e-03],
          [-7.1164e-04, -2.8363e-04,  2.3546e-04,  ...,  2.6077e-03,
            5.8768e-04,  1.7746e-02],
          [ 1.5474e-03, -8.1548e-04,  3.7796e-04,  ...,  3.0229e-03,
            9.3371e-03,  4.8188e-03]],

         [[-4.1022e-04, -3.4290e-05, -3.0182e-04,  ...,  8.9021e-04,
           -4.7598e-03,  3.3510e-03],
          [-2.5946e-04, -5.1724e-04,  1.9255e-04,  ...,  6.7862e-04,
           -4.9981e-05,  5.1331e-04],
          [-4.8008e-04, -8.8690e-05,  9.4650e-05,  ...,  1.8057e-03,
           -4.2970e-05,  3.6180e-03],
          ...,
     

In [10]:
gt_data

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')